In [ ]:
!pip install requests pyvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.8 MB/s eta 0:00:00


In [ ]:
import requests
from pyvis.network import Network
import datetime
import numpy as np

ETHERSCAN_API_KEY = 'R82AC7Z4WQ6BDP5T92SEICH8U4VJ2DJUXS'

ethereum_address = input("Please enter the Ethereum address: ")

def get_transactions(address):
    url = f"https://api.etherscan.io/api?module=account&action=txlist&address={address}&startblock=0&endblock=99999999&sort=asc&apikey={ETHERSCAN_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()['result']
    else:
        print("Error fetching transactions")
        return []


transactions = get_transactions(ethereum_address)
processed_transactions = process_transactions(transactions)

# Heuristics to detect potential fraud
def analyze_transactions_for_fraud(transactions):
    timestamps = [tx[3] for tx in transactions]
    values = [tx[4] for tx in transactions]
    if timestamps:
        # Calculate the average time between transactions
        average_delta = np.mean(np.diff(sorted(timestamps))).total_seconds()
        # Flag if the average time between transactions is suspiciously short
        is_suspicious_timing = average_delta < 60  # less than a minute

        # Identify large transactions
        large_transactions = [tx for tx in transactions if tx[4] > 1]  # Over 50 Ether

        # Check if address was active for a short time but with a high volume of transactions
        active_period = (max(timestamps) - min(timestamps)).total_seconds()
        is_short_lived = active_period < (24 * 60 * 60) and len(transactions) > 100  # Active less than a day but more than 100 transactions

        fraud_indicators = {
            'is_suspicious_timing': is_suspicious_timing,
            'large_transactions': large_transactions,
            'is_short_lived': is_short_lived,
        }
        return fraud_indicators
    return {}

fraud_analysis = analyze_transactions_for_fraud(processed_transactions)

def create_interactive_graph(transactions, address, fraud_analysis):

    net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", notebook=True, cdn_resources='remote')


    for tx_hash, from_addr, to_addr, timestamp, value_ether in transactions:
        title = f"Hash: {tx_hash}\nFrom: {from_addr}\nTo: {to_addr}\nTimestamp: {timestamp}\nAmount: {value_ether} ETH"
        net.add_node(from_addr, title=from_addr, color='#dd4b39')
        net.add_node(to_addr, title=to_addr)
        net.add_edge(from_addr, to_addr, title=title, value=value_ether)

    # Highlight large transactions with a different color or width
    for tx in fraud_analysis.get('large_transactions', []):
        title = f"Large Transaction (Over 50 ETH)! Hash: {tx[0]}\nFrom: {tx[1]}\nTo: {tx[2]}\nTimestamp: {tx[3]}\nAmount: {tx[4]} ETH"
        net.add_edge(tx[1], tx[2], title=title, value=tx[4], color='red')


    net.force_atlas_2based()
    return net

interactive_graph = create_interactive_graph(processed_transactions, ethereum_address, fraud_analysis)

interactive_graph.show("ethereum_transactions.html")


Please enter the Ethereum address: 0xef57aa4b57587600fc209f345fe0a2687bc26985
ethereum_transactions.html
